streaming bronze tables from data lake {source}

In [ ]:
%%sql
CREATE STREAMING LIVE TABLE stream_bronze_device
COMMENT "streaming from data lake"
TBLPROPERTIES ("Quality" = "Bronze")
AS SELECT * FROM cloud_files("dbfs:/FileStore/tables/pipeline/landing/device/", "json", map("cloudFiles.inferColumnTypes", "true"));

CREATE STREAMING LIVE TABLE stream_bronze_subscription
COMMENT "streaming from data lake"
TBLPROPERTIES ("Quality" = "Bronze")
AS SELECT * FROM cloud_files("dbfs:/FileStore/tables/pipeline/landing/subscription/", "json", map("cloudFiles.inferColumnTypes", "true"));

create silver table {domain table}

In [ ]:
%%sql
CREATE STREAMING LIVE TABLE stream_silver_subscriptions(
  CONSTRAINT valid_user_id EXPECT (user_id IS NOT NULL) ON VIOLATION DROP ROW
)
PARTITIONED BY (user_id)
COMMENT "subscription domain table, cleansed data from device and subscription bronze tables."
TBLPROPERTIES ("Quality" = "Silver")
AS
SELECT 
  d.user_id,
  d.model,
  d.manufacturer,
  d.platform,
  s.plan,
  s.status
FROM STREAM(LIVE.stream_bronze_device) AS d
INNER JOIN LIVE.stream_bronze_subscription AS s
ON d.user_id = s.user_id